In [1]:
from WorkforceSentimentMonitoring.preprocessing import preprocessing
from WorkforceSentimentMonitoring.data import get_data, merge
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from langdetect import detect

In [2]:
submission, train, test = get_data("/Users/theresasporn/code/theresa-sporn/workforce_sentiment_monitoring/raw_data")

In [14]:
text = merge(submission, train, test)
#positives = text['positives']
#positives.head(5)
text.head()

,summary,positives,negatives,advice_to_mgmt,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,review
0,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,NaN,4,5,5,4,5,5,Best Company to work for People are smart and ...
1,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,2,3,3,5,3,5,"Moving at the speed of light, burn out is inev..."
2,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,5,4,5,5,4,5,Great balance between big-company security and...
3,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,2,5,5,4,5,4,The best place I've worked and also the most d...
4,Execellent for engineers,Impact driven. Best tech in the world.,Size matters. Engineers are a bit disconnected...,NaN,5,5,5,5,5,4,Execellent for engineers Impact driven. Best t...


In [87]:
df = pd.read_csv("/Users/theresasporn/Downloads/pos_neg_df.csv")
df.head()

,positives,negatives
0,work people passionate care greater good fanta...,need open office many office sunlight
1,lot knowledge flexible hour office process rea...,key problem company lost true leader existing ...
2,immense carrier opportunity growth opportunity...,working fc first difficult week get habitual s...
3,employee food benefit perk shuttle basically e...,company become way vertical performance evalua...
4,good pay somewhat nice people,hired ’ always something


In [92]:
type(df.negatives[0])

str

In [26]:
def extract_negative(df):
    """return df with negative reviews and their labels"""
    negatives = df[['negatives']]
    #negatives.loc[:,'sentiment'] = 0 # 0=> negative
    return negatives

In [27]:
negatives = extract_negative(df)
negatives.head(5)

,negatives
0,need open office many office sunlight
1,key problem company lost true leader existing ...
2,working fc first difficult week get habitual s...
3,company become way vertical performance evalua...
4,hired ’ always something


In [94]:
from nltk import word_tokenize
def tokenize(df):
    tokenized_text = word_tokenize(str(df))
    return tokenized_text

In [105]:
negatives_tokenized = df['negatives'].apply(tokenize)
negatives_tokenized

0             [need, open, office, many, office, sunlight]
1        [key, problem, company, lost, true, leader, ex...
2        [working, fc, first, difficult, week, get, hab...
3        [company, become, way, vertical, performance, ...
4                            [hired, ’, always, something]
5        [pay, low, opportunity, move, within, pay, sca...
6        [lot, lot, meeting, top, decision, making, lot...
7        [environment, hyper, competitive, everybody, r...
8        [management, matured, always, end, micro, mana...
9                                           [amazon, scam]
10       [difficult, work, environment, useless, compet...
11                        [office, politics, slow, career]
12       [discrimination, back, stabbing, retaliation, ...
13       [working, ar, site, suck, oak, micromanaged, t...
14       [job, role, limited, hard, get, promotion, sal...
15       [hardly, decency, mutual, respect, among, empl...
16       [amount, work, rapid, changing, environment, p.

In [28]:
def extract_positive(df):
    """return df with positive reviews and their labels"""
    positives = df[['positives']]
    #text.loc[:,'sentiment'] = 1 # 1=> positive
    return positives

In [29]:
positives = extract_positive(df)
positives.head(5)

,positives
0,work people passionate care greater good fanta...
1,lot knowledge flexible hour office process rea...
2,immense carrier opportunity growth opportunity...
3,employee food benefit perk shuttle basically e...
4,good pay somewhat nice people


In [113]:
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from gensim.corpora import Dictionary
import pyLDAvis
import pyLDAvis.gensim

id2word = corpora.Dictionary(negatives_tokenized)
dictionary = id2word
texts = negatives_tokenized
corpus = [id2word.doc2bow(text) for text in texts]
ldamallet = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=2, id2word=id2word, iterations=100)
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()

    
def compute_coherence_values(dictionary, corpus, texts, limit=8, start=2, step=1):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word, alpha=.91)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=4, limit=8, step=2)


max_y = max(coherence_values)  # Find the maximum y value
max_x = coherence_values.index(max(coherence_values))  # Find the x value corresponding to the maximum y value
#xmax = x[numpy.argmax(y)]

optimal_model = model_list[coherence_values.index(max(coherence_values))]

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)
vis


/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.102596 -0.119572       1        1  22.440048
3      0.046445 -0.060070       2        1  17.064289
2      0.091545  0.028912       3        1  16.847029
5      0.075185  0.071451       4        1  16.511112
1      0.040805 -0.028591       5        1  14.700227
0     -0.151384  0.107870       6        1  12.437291, topic_info=     Category          Freq         Term         Total  loglift  logprob
179   Default  12827.000000         work  12827.000000  30.0000  30.0000
214   Default   4212.000000         hour   4212.000000  29.0000  29.0000
177   Default   4624.000000         team   4624.000000  28.0000  28.0000
133   Default   2584.000000         long   2584.000000  27.0000  27.0000
249   Default   2651.000000      balance   2651.000000  26.0000  26.0000
253   Default   3437.000000         life   3437.000000  25.0000  25.0000
536   Default   3190.000000         year   3190.000000  24.0000  24.0000
453   Default   5825.000000      manager   5825.000000  23.0000  23.0000
360   Default   6858.000000         time   6858.000000  22.0000  22.0000
108   Default   1963.000000    sometimes   1963.000000  21.0000  21.0000
212   Default   1922.000000         high   1922.000000  20.0000  20.0000
127   Default   1426.000000            ’   1426.000000  19.0000  19.0000
996   Default   1656.000000        apple   1656.000000  18.0000  18.0000
22    Default   7047.000000      company   7047.000000  17.0000  17.0000
278   Default   3189.000000          day   3189.000000  16.0000  16.0000
153   Default   1504.000000  environment   1504.000000  15.0000  15.0000
205   Default   1200.000000        break   1200.000000  14.0000  14.0000
1104  Default   1438.000000       retail   1438.000000  13.0000  13.0000
195   Default   6738.000000       people   6738.000000  12.0000  12.0000
229   Default   1511.000000        shift   1511.000000  11.0000  11.0000
980   Default   1662.000000       review   1662.000000  10.0000  10.0000
625   Default   1696.000000         want   1696.000000   9.0000   9.0000
109   Default   2370.000000        thing   2370.000000   8.0000   8.0000
13    Default   1898.000000          bad   1898.000000   7.0000   7.0000
1137  Default   1188.000000      project   1188.000000   6.0000   6.0000
919   Default   1110.000000          con   1110.000000   5.0000   5.0000
51    Default   6174.000000   management   6174.000000   4.0000   4.0000
539   Default   2521.000000     customer   2521.000000   3.0000   3.0000
1118  Default    757.000000         fast    757.000000   2.0000   2.0000
450   Default   1335.000000   leadership   1335.000000   1.0000   1.0000
...       ...           ...          ...           ...      ...      ...
477    Topic6    476.540161     pressure    752.162598   1.6281  -5.1335
3210   Topic6    168.114365         load    231.542908   1.7644  -6.1755
179    Topic6   4959.272949         work  12827.126953   1.1342  -2.7911
919    Topic6    614.245300          con   1110.657227   1.4922  -4.8797
984    Topic6    310.716461       stress    496.215759   1.6163  -5.5612
1268   Topic6    273.102997         peak    437.288696   1.6137  -5.6902
1261   Topic6    331.281433      holiday    568.285278   1.5448  -5.4971
1823   Topic6    174.493149       demand    257.642883   1.6948  -6.1382
205    Topic6    586.413391        break   1200.564087   1.3679  -4.9261
799    Topic6    138.612640     standing    193.889374   1.7489  -6.3684
729    Topic6    387.917023  expectation    729.675415   1.4527  -5.3393
893    Topic6    238.549393         busy    406.189636   1.5522  -5.8255
87     Topic6    585.911377    difficult   1439.312256   1.1857  -4.9269
360    Topic6   1753.504150         time   6858.235352   0.7206  -3.8307
243    Topic6    929.428833         hard   2793.505371   0.9840  -4.4655
134    Topic6    473.459900          low   1075.454346   1.2640  -5.1400
135    Topic6    735.

In [ ]:
pyLDAvis.save_html(pyLDAvis.gensim.prepare(ldamallet, corpus, id2word), 'pyLDAvis_negatives_tokenized.html')
#Where 'vis' is the name of your model, like this:
#vis = pyLDAvis.gensim.prepare(ldamallet, corpus, id2word)

/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [112]:
vis

/Users/theresasporn/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.199139 -0.023159       1        1  12.473089
12     0.034372 -0.085708       2        1   9.089458
6      0.001850 -0.093952       3        1   8.269097
7     -0.037427 -0.047676       4        1   7.953890
13    -0.080359 -0.003753       5        1   7.724688
0      0.074027 -0.044417       6        1   7.043749
11     0.137967  0.026006       7        1   6.632311
8      0.036776 -0.040993       8        1   6.588508
4      0.096529  0.020148       9        1   6.012059
2     -0.023347  0.102681      10        1   5.886488
5      0.017094  0.041534      11        1   5.813758
10    -0.009718  0.029615      12        1   5.583517
9     -0.033602  0.029181      13        1   5.481916
1     -0.015024  0.090495      14        1   5.447462, topic_info=     Category          Freq         Term         Total  loglift  logprob
179   Default  13730.000000         work  13730.000000  30.0000  30.0000
253   Default   3717.000000         life   3717.000000  29.0000  29.0000
249   Default   2873.000000      balance   2873.000000  28.0000  28.0000
539   Default   2711.000000     customer   2711.000000  27.0000  27.0000
1104  Default   1567.000000       retail   1567.000000  26.0000  26.0000
127   Default   1489.000000            ’   1489.000000  25.0000  25.0000
996   Default   1880.000000        apple   1880.000000  24.0000  24.0000
1054  Default    981.000000        store    981.000000  23.0000  23.0000
177   Default   4967.000000         team   4967.000000  22.0000  22.0000
22    Default   7411.000000      company   7411.000000  21.0000  21.0000
98    Default   2078.000000        level   2078.000000  20.0000  20.0000
536   Default   3218.000000         year   3218.000000  19.0000  19.0000
205   Default   1161.000000        break   1161.000000  18.0000  18.0000
46    Default   4318.000000          job   4318.000000  17.0000  17.0000
214   Default   4336.000000         hour   4336.000000  16.0000  16.0000
51    Default   6531.000000   management   6531.000000  15.0000  15.0000
180   Default   1730.000000     politics   1730.000000  14.0000  14.0000
278   Default   3141.000000          day   3141.000000  13.0000  13.0000
243   Default   2988.000000         hard   2988.000000  12.0000  12.0000
145   Default   4134.000000          lot   4134.000000  11.0000  11.0000
229   Default   1458.000000        shift   1458.000000  10.0000  10.0000
896   Default    800.000000         sale    800.000000   9.0000   9.0000
40    Default   6920.000000          get   6920.000000   8.0000   8.0000
245   Default   1438.000000    promotion   1438.000000   7.0000   7.0000
219   Default    689.000000       minute    689.000000   6.0000   6.0000
174   Default   2898.000000       amazon   2898.000000   5.0000   5.0000
453   Default   6077.000000      manager   6077.000000   4.0000   4.0000
108   Default   2140.000000    sometimes   2140.000000   3.0000   3.0000
195   Default   7171.000000       people   7171.000000   2.0000   2.0000
129   Default   1808.000000       career   1808.000000   1.0000   1.0000
...       ...           ...          ...           ...      ...      ...
108   Topic14    506.836487    sometimes   2140.230713   1.4695  -4.2464
2555  Topic14     48.046413         loud     81.031342   2.3874  -6.6024
145   Topic14    665.578491          lot   4134.313477   1.0836  -3.9739
133   Topic14    462.233765         long   2759.057129   1.1234  -4.3385
972   Topic14    236.796158         done    995.685547   1.4738  -5.0073
1     Topic14    420.299957         need   2408.564941   1.1642  -4.4336
40    Topic14    812.464722          get   6920.098145   0.7679  -3.7745
179   Topic14   1149.446411         work  13730.755859   0.4297  -3.4275
729   Topic14    186.877762  expectation    784.623230   1.4753  -5.2441
243   Topic14    408.927460         hard   2988.322754   0.9211  -4.4610
1628  Topic14    149.140381